In [1]:
import os
import requests
import pandas as pd
import numpy as np
from concurrent.futures import ThreadPoolExecutor

In [2]:
data = pd.read_csv('./ukiyo-e_crawler/meta_data/ukiyo-e_artworks_details.csv')
data.head()

,Image URL,Artist,Title,Date,Details,Source,Description,Similar Prints
0,https://data.ukiyo-e.org/bm/scaled/AN00418884_...,Okumura Masanobu,NaN,1711,https://www.britishmuseum.org/collection/objec...,http://www.britishmuseum.org/research/search_t...,Print. Sixth of series of twelve (first two mi...,[]
1,https://data.ukiyo-e.org/mfa/scaled/sc220382.jpg,Okumura Masanobu,Actor Sanogawa Ichimatsu as a Falconer,1740,https://collections.mfa.org/objects/224230,http://www.mfa.org/search/collections,NaN,['https://ukiyo-e.org/image/bm/AN00611916_001_l']
2,https://data.ukiyo-e.org/loc/scaled/02127v.jpg,Okumura Masanobu,Courtesan likened to the Chinese sage Zhang Gu...,"[1715, printed later]",http://www.loc.gov/pictures/collection/jpd/ite...,http://www.loc.gov/pictures/search/?sp=2&co=jp...,Print shows a courtesan portraying the Chinese...,[]
3,https://data.ukiyo-e.org/bm/scaled/AN00420719_...,Okumura Masanobu,NaN,1686-1764 (artist),https://www.britishmuseum.org/collection/objec...,http://www.britishmuseum.org/research/search_t...,Woodblock print. Kabuki. The actor Sanjo Kanta...,[]
4,https://data.ukiyo-e.org/mfa/scaled/sc200151.jpg,Okumura Masanobu,"Tsuri ---Ki no sendaku, from an untitled serie...","Japanese, Edo period",https://collections.mfa.org/objects/256153,http://www.mfa.org/search/collections,NaN,[]


In [3]:
data = data.dropna(subset=['Image URL'])

In [4]:
data['Image URL'] = data['Image URL'].apply(lambda x: x.replace('scaled', 'images'))

In [13]:
def download_image(url):
    try:
        path = './ukiyo-e_crawler/images/' + url.split('/')[-1][0:-4] + '.jpg'
        response = requests.get(url)
        with open(path, 'wb') as f:
            f.write(response.content)
    except:
        print('error: ', url)

urls = remaining

with ThreadPoolExecutor(max_workers=7) as executor:
    executor.map(download_image, urls)

In [8]:

def check_download(url):
    try:
        path = './ukiyo-e_crawler/images/' + url.split('/')[-1][0:-4] + '.jpg'
        if os.path.exists(path):
            return(url, True) 
        else:
            return(url, False) 
    except:
        print('error: ', url)

urls = data['Image URL']
results = []

with ThreadPoolExecutor(max_workers=10) as executor:
    for result in executor.map(check_download, urls):
        results.append(result)

check = pd.DataFrame(results, columns=['url','downloaded'])


In [9]:
check.head()

,url,downloaded
0,https://data.ukiyo-e.org/bm/images/AN00418884_...,True
1,https://data.ukiyo-e.org/mfa/images/sc220382.jpg,True
2,https://data.ukiyo-e.org/loc/images/02127v.jpg,True
3,https://data.ukiyo-e.org/bm/images/AN00420719_...,True
4,https://data.ukiyo-e.org/mfa/images/sc200151.jpg,True


In [12]:
remaining = check['url'][~check['downloaded']]
remaining.head()

41     https://data.ukiyo-e.org/artofjapan/images/ccf...
207    https://data.ukiyo-e.org/artofjapan/images/ccf...
275    https://data.ukiyo-e.org/artofjapan/images/8e1...
364    https://data.ukiyo-e.org/artofjapan/images/d58...
441    https://data.ukiyo-e.org/artofjapan/images/50a...
Name: url, dtype: object